# Problem Statement
Performing an efficient management of budget and securing accuracy are key points when using crowdsourcing. In classification scenarios the crowd sometimes can be noisy and generate a lack of concensus over tasks. The majority of the state-of-art approaches set a fixed number of votes per item and aggregate the votes with some criteria, this produces a reduction in the quality and an increment of costs. For this reason we present a smart detection algorithm which predicts the best decision between stop or continue collecting votes over a task and analyze its efficiency with different experiments.

Formalizing the problem, given a set of items $I$, a set of votes $V$, a classification function $fn$, a classification threshold $th$ and a cost ratio for crowd to expert vote cost $cr$, for each item we want to find the minimum amount of votes needed to take the decision of continue collecting votes or switch to an expert vote. For this we describe a smart stopping algorithm. 

**We define a 3 methods structure**:
- the **classifier** which returns the probability of an item being classified
- the **cost estimator** which returns the estimated cost for each item given the votes
- the **decision function** which returns a boolean decision for each item

# Rationale for the algorithms

### Classifier function

- Input: item votes
- Output: probability IN [0, 1]

### Cost estimator function


- Input: item votes, classification threshold, classification function, cost ratio
- Output: predicted cost
- Logic:
    - Iterate N times for converging the results
        - Calculate actual cost for the given votes
        - While (is not classified or is not too expensive)
            - If actual cost is not too expensive
                - Get P(IN) using the classification function
                - If P(IN) > threshold or P(OUT) > threshold
                    - **Stop because item is classified**
                - Else
                    - Simulate 1 vote over the item using P(IN)
                    - Increment actual cost
            - Else **stop because the item is too expensive to be classified**

### Decision function



- Input: items, votes, classification_threshold, cost_ratio, classification_function
- Output: Boolean decision over each item, where True = continue and False = stop collecting votes
- Logic:
    - For each item
        - Get P(IN) using the classification function
        - If P(IN) > classification threshold or P(OUT) > classification threshold
            - Decision over item = False
        - Else
            - Call **cost estimator function**
                - If cost estimated <= max cost
                    - Decision over item = True
                - Else
                    - Decision over item = False


### Item Classification without expert vote

* The logic for item classification once votes are collected is defined as:
    * P(i = OUT|item_votes) = 1 - classifier_function(item_votes)
    * If P(i = OUT|item_votes) > classification_threshold -> item is classified as OUT **else** is classified as IN
    
    
    
- This can be interpreted as if there isn't enough evidence(votes) to classify the item OUT, is classified IN. This directly affects the Recall metric due to the false positives.

# Experiments

Here we discuss a few experiments, the objective is to compare the overall crowdsourcing cost and quality in the case where we have a smart stopping algorithm vs 
- the baseline approach where all items receive the same amount of votes
- an approach used by Andrew W. Brown and David B. Allison. 2014. Using Crowdsourcing to Evaluate Published Scientific Literature: Methods and Example. Plos One 9, 7 (2014). Where 2 votes are requested and if they disagree ask a third to break the tie

Also we evaluate the performance using a smart stop detection mechanism in terms of cost and quality for balanced and unbalanced datasets, and in different expert costs.

In [14]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import helpers.algorithms_utils as alg_utils
from IPython.display import HTML
import matplotlib.pyplot as plt
from IPython.core import display as ICD
from helpers.mv_single_binary import majority_voting
from helpers.truth_finder import expectation_maximization
from helpers.truth_finder_single import truth_finder_single
from algorithms.abraham_stop import abraham_stop_binary
from algorithms.smart_stop import decision_function_mv
from algorithms.smart_stop import decision_function_em
from algorithms.smart_stop import decision_function_bayes
import multiprocessing

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### Hyperparameters for all experiments

 - Classification function: MV, EM
 - Cost Ratio: [0,1]
 - Workers: 
     - Amount: at least (1/cost_ratio) * increment_parameter
     - Percentage of cheaters: [0,1]
     - Distribution: [0,1]
     - Fixed accuracy for all workers: Boolean
 - Data:
     - Amount: > 0
     - Ground Truth Balance(percentage of positive items): [0,1]
 - Experiment convergence:
     - Amount of iterations: > 0
 - Classification:
     - Thresholds: [0,1]
 - Metrics:
     - Loss ratio(penalty for False Negatives): > 0 
 

In [4]:
#main
'''
cf = majority_voting
cr = .05
base_votes_per_item = 3

#cost estimator
drawing_simulations_amount = 50
expert_cost_increment = 2

#crowd
workers_num = 2000 #fixed number
z = 0 #% cheaters
base_workers_acc = .5
fixed_acc = False
fixed_workers_acc = .9


#ground truth 
items_num = 1000
data_true_percentage = .5

#experiment 
iterations_per_ct = 50
cts = [.5, .6, .7, .8, .9] #classification thresholds

#loss ratio: false negative increment penalization
lr = 5 #1 means not penalization
'''
#alg_utils.print_hyperparameters(cf, cr, base_votes_per_item, drawing_simulations_amount, expert_cost_increment, workers_num, z, fixed_acc, base_workers_acc, fixed_workers_acc, items_num, data_true_percentage, iterations_per_ct, cts, lr)

pdColumns = ["loss_ratio", "cost_ratio", "class_fn", "decision_fn", "data_bal", "threshold", "cost","cost_std", "loss", "loss_std", "recall", "recall_std", "precision", "precision_std"]

## Datasets

### Single-predicate classification
   - **RTE**: 800 items. 
     - Includes binary judgments for textual entailment. 
   - **TEMP**: 462 items. 
     - Includes binary judgments for temporal ordering (i.e., whether one event follows another).
   - **BM**: 1000 items. 
     - (Mozafari et al. 2012) contains negative/positive sentiment labels assigned by AMT workers to tweets.
   - **SpamCF**: 100 items. 
     - (Ipeirotis 2010) includes binary AMT judgments about whether or not an AMT HIT should be considered a “spam” task, according to their criteria.
   - **WVSCM**: 159 items. 
     - (Whitehill et al. 2009) includes AMT binary judgments distinguishing whether or not face images smile.

### Multi-predicate classification
   - **ohsumed-based screening dataset**: 34k Two-predicate screening datasets presented by medical abstracts on different disease categories. 
   - **loneliness-slr-2018**: The dataset includes 585 abstracts of papers annotated by an expert for 2 predicates. The papers are from social informatics domain and used as a part of the systematic literature review (SLR). 
   - **amazon-sentiment-dataset**: 1k-100k dataset on Amazon product reviews, where predicats are 1) Product category 2) Sentiment of the review.
     

In [3]:
datasets = [
    {'folder':'amazon-sentiment-dataset', 'filename':'1k_amazon_reviews_crowdsourced_lemmatized_min3votes.csv','predicates': ['is_negative', 'is_book']},
    {'folder':'loneliness-dataset-2018', 'filename':'loneliness-dataset-2018.csv','predicates': ['oa_predicate', 'study_predicate']},
    {'folder':'ohsumed_data', 'filename':'ohsumed_C14_C23_1grams.csv','predicates': ['C14', 'C23']},
    {'folder':'BarzanMozafari', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
    {'folder':'RTE', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
    {'folder':'SpamCF', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
    {'folder':'TEMP', 'filename':'ground_truth_normalized.csv','predicates':['gt']}, 
    {'folder':'WVSCM', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
]

In [5]:
def run_experiment(generate_votes_fn, pool, df, cf, data_true_percentage):
    main_results = []
    
    iterables = []
    for ct in cts:
        multi = 5
        divided_it_per_process = int(iterations_per_ct / multi)
        for _ in range(multi):
            iterables.append((divided_it_per_process, ct, generate_votes_fn, df, cr, cf, 
                              workers_num, z, fixed_acc, fixed_workers_acc, base_workers_acc,
                              base_votes_per_item, ground_truth, data_true_percentage, items_num
                             ))
        
        
        results = pool.map(iterate_ct, iterables)
        
        main_results.append([lr,
                             cr,
                             str(cf.__name__),
                             str(df.__name__),
                             data_true_percentage,
                             ct, 
                             round(np.mean([x[6] for x in results]), 3), 
                             round(np.mean([x[7] for x in results]), 3),
                             round(np.mean([x[8] for x in results]), 3),
                             round(np.mean([x[9] for x in results]), 3),
                             round(np.mean([x[10] for x in results]), 3),
                             round(np.mean([x[11] for x in results]), 3),
                             round(np.mean([x[12] for x in results]), 3),
                             round(np.mean([x[13] for x in results]), 3)])       
        
    
    return main_results


In [6]:
def iterate_ct(parameters):
        iterations, ct, generate_votes_fn, decision_fn, cr, cf, \
                              workers_num, z, fixed_acc, fixed_workers_acc, base_workers_acc, \
                              base_votes_per_item, ground_truth, data_true_percentage, items_num  = parameters
        
        ct = round(ct, 2) #limit to two decimals
        crowd_cost = []
        ct_loss = []
        ct_recall = []
        ct_precision = []
        
        for _ in range(iterations):
            workers_accuracy = alg_utils.simulate_workers(workers_num, z, fixed_acc, fixed_workers_acc, base_workers_acc)
            
            params = {
                'workers_accuracy': workers_accuracy,
                'workers_num': workers_num,
                'items_num': items_num,
                'cost_ratio': cr,
                'votes_per_item': base_votes_per_item,
                'classification_fn': cf
            }
    
            items_classification, total_votes = generate_votes_fn(params, items_num, ct, ground_truth, decision_fn)
            
            loss,  recall, precision = alg_utils.Metrics.compute_metrics(items_classification, ground_truth, lr)
            ct_loss.append(loss)
            ct_recall.append(recall)
            ct_precision.append(precision)
            
            ct_i_crowd_cost = alg_utils.get_total_cost(total_votes, cr)
            
            crowd_cost.append(ct_i_crowd_cost)
        #end for iterations

        return [lr,
             cr,
             str(cf.__name__),
             str(decision_fn.__name__),
             data_true_percentage,
             ct, 
             round(np.mean(crowd_cost), 3), 
             round(np.std(crowd_cost), 3),
             round(np.mean(ct_loss), 3),
             round(np.std(ct_loss), 3),
             round(np.mean(ct_recall), 3),
             round(np.std(ct_recall), 3),
             round(np.mean(ct_precision), 3),
             round(np.std(ct_precision), 3)
            ]

In [7]:
def run_experiment_no_multiprocessing(generate_votes_fn, pool, df, cf, data_true_percentage):
    
    main_results = []
    
    for ct in cts:
        ct = round(ct, 2) #limit to two decimals
        crowd_cost = []
        ct_loss_mv = []
        ct_recall_mv = []
        ct_precision_mv = []
        
        for _ in range(iterations_per_ct):
            workers_accuracy = alg_utils.simulate_workers(workers_num, z, fixed_acc, fixed_workers_acc, base_workers_acc)
            
            params = {
                'workers_accuracy': workers_accuracy,
                'workers_num': workers_num,
                'items_num': items_num,
                'cost_ratio': cr,
                'votes_per_item': base_votes_per_item,
                'classification_fn': cf
            }
    
            items_classification, total_votes = generate_votes_fn(params, items_num, ct, ground_truth, df)
            
            loss,  recall, precision = alg_utils.Metrics.compute_metrics(items_classification, ground_truth, lr)
            ct_loss_mv.append(loss)
            ct_recall_mv.append(recall)
            ct_precision_mv.append(precision)     
            
            ct_i_crowd_cost = alg_utils.get_total_cost(total_votes, cr)
            crowd_cost.append(ct_i_crowd_cost)
        #end for iterations

        main_results.append([lr,
                             cr,
                             str(cf.__name__),
                             str(df.__name__),
                             data_true_percentage,
                             ct, 
                             round(np.mean(crowd_cost), 3), 
                             round(np.std(crowd_cost), 3),
                             round(np.mean(ct_loss_mv), 3),
                             round(np.std(ct_loss_mv), 3),
                             round(np.mean(ct_recall_mv), 3),
                             round(np.std(ct_recall_mv), 3),
                             round(np.mean(ct_precision_mv), 3),
                             round(np.std(ct_precision_mv), 3)])

        
    
    return main_results


## 1) Experiments without considering workers accuracy for predictions

## 1.1) Smart stop with MV as classification function, where the Expected cost is limited by expert cost.

## 1.1.1) Logic

1. Evalute for each classification threshold
2. Iterate N times per threshold for converging results doing:
    1. Generate workers accuracy
    2. Generate ground truth values
    3. Generate the base votes for each item
    4. Call the decision function
        1. If we have any item to continue collecting votes:
            1. Collect 1 more vote per item
            2. Call decision function
    5. Classify items using MV
        1. If P(OUT) > threshold, classify it as OUT
        2. Else classify it as IN
    6. Calculate the metrics

In [8]:
def generate_votes_smart_stop(params, items_num, ct, gt, decision_fn):
    total_votes = {}
    
    #base votes
    for i in range(items_num):
        total_votes[i] = {}
        for k in range(params['votes_per_item']):
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)

            total_votes[i][worker_id] = [vote]
            
    #evaluate votes
    results = decision_fn(items_num, total_votes, ct, params['cost_ratio'], 
                                                   params['classification_fn'])
    
    #Check if must continue collecting votes
    items_predicted_classified = alg_utils.get_items_predicted_classified(results)
    must_get_more_votes = len(items_predicted_classified) > 0
     
    while(must_get_more_votes):
        total_votes_aux = {}
        for i, v in items_predicted_classified.items():           
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)
            
            total_votes[i][worker_id] = [vote]
            
            total_votes_aux[i] = total_votes[i].copy()
        #end for
        
        results = decision_fn(len(total_votes_aux), total_votes_aux, ct, params['cost_ratio'], 
                                                   params['classification_fn'])

        #Stop when there are no more items that can be classified
        items_predicted_classified = alg_utils.get_items_predicted_classified(results)
        must_get_more_votes = len(items_predicted_classified) > 0
    #end while
    
    items_40 = len([1 for k,v in total_votes.items() if len(v) >= 40])
    items_3 = len([1 for k,v in total_votes.items() if len(v) == 3])
    rest = len(total_votes) - items_40 - items_3
    print(f'Items3: {items_3} - Items40: {items_40} - ItemsRest: {rest}')
    
    items_classification = alg_utils.classify_items(total_votes, gt, majority_voting, .5)

    return items_classification, total_votes

## 1.2) MV with fixed number of votes per item

In this experiment we use the baseline approach of majority voting with a fixed number of votes per item and a classification threshold of .5

## 1.2.1) Logic

1. Iterate N times for converging results where:
    1. Generate workers accuracy
    2. Generate ground truth values
    3. Generate a fixed number of votes for each item
    4. Classify items using MV
    5. Calculate the metrics

In [42]:
def generate_votes_mv_base(params, items_num, ct, gt, decision_fn=None):
    total_votes = {}

    #base votes
    for i in range(items_num):
        total_votes[i] = {}
        for k in range(params['votes_per_item']):
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)

            total_votes[i][worker_id] = [vote]

    items_classification = alg_utils.classify_items(total_votes, gt, params['classification_fn'], ct)
    
    return items_classification, total_votes

## 1.3) Two votes first and if disagree one more to break the tie

In this experiment we collect 2 votes per item and a third one if they disagree.

## 1.3.1) Logic

1. Iterate N times for converging results where:
    1. Generate workers accuracy
    2. Generate ground truth values
    3. Generate 2 votes for each item
    4. Evalutes the votes:
        1. If votes agree:
            1. Jump to next step
        2. If votes disagree:
            1. Get 1 more vote 
    4. Classify items using MV
    5. Calculate the metrics

In [43]:
def generate_votes_break_tie(params, items_num, ct, gt, decision_fn=None):
    total_votes = {}

    #base votes
    for i in range(items_num):
        total_votes[i] = {}
        for k in range(2):
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)

            total_votes[i][worker_id] = [vote]
            
        #if disagree get 1 more vote
        votes_disagree = sum([v[0] for i,v in total_votes[i].items()]) == 1
        if(votes_disagree):
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)

            total_votes[i][worker_id] = [vote]

    items_classification = alg_utils.classify_items(total_votes, gt, params['classification_fn'], ct)
    
    return items_classification, total_votes

# 3.3) Results evaluation

## Experiments with simulated balanced data

##  Analysis over different classification thresholds using different expert costs

### Smart stop over different expert costs

The expert cost has a direct impact in the experiments overall cost, in particular for our approach the expert cost sets a limit for the amount of drawings we do for simulate votes. A lower expert cost means a lower cost ratio between crowd to expert cost, and this is translated in the amount of crowd votes we can collect until reaching the expert.  This is one of the key points in our analysis and this is to choose the best alternative between collecting more votes or not to be more confident for taking a decision for classifying an item. 


To observe how the expert cost impact in the performance we set different costs: 10, 50, 100 and 500. To clarify the notion this can be read easily as a relation, if our expert cost is 10 and the crowd cost is 1 we have to collect 10 crowd votes to reach the expert cost, and can be seen as a ratio 1:10.

to-do clarify:
Expert cost -> # crowd votes -> prob -> classification threshold


**SmartStop approach behaviour using MV. Simulations for balanced data, over different classifications threshold = .7, with 1000 balanced items, expert costs 10, 20, 50 and 100, workers acc mean ≈ 76%, loss ratio = 5.**

In the graphics we can observe that the cost ratio don't affect the quality. This was expected given the prior analysis of the behaviour of majority voting, when having collected 3 votes which only 2 agree MV gives us a 66,6 probability of being classified as one class, and if we collect 1 more this can turn this probability in 75% if agree and gets classified, or 50% if disagree turning it in what we assume the worse case for drawing because represent random voting.

### Smart stop over different classification thresholds with fixed expert cost

One point to see is the performance of the approach using the smart stop mechanism over different classification thresholds with a fixed expert cost, with this we expect to see the behaviour between the classification function, the threshold and the given cost.

**SmartStop approach behaviour using MV. Simulations for balanced data, over different classifications threshold = [.5, .6, .7, .8, .9], with 1000 balanced items, expert cost 20, workers acc mean ≈ 76%, loss ratio = 5.**

### Cost vs loss

***SmartStop approach performance analysis:***

 - We can observe for the classification thresholds .5 and .6 the same behaviour given the percentage obtained by MV when 3 votes are collected. When the 3 votes collected agree MV gives us a 100% probability of being classified as 1 of the 2 classes, either 0 or 1, instead when only 2 votes agree we get a 66,6% chance of classification over one class and this reachs .5 and .6 thresholds acting as the baseline logic. 

 - For threshold .7 we observe that many votes were collected(**1096** more than base) which incremented the quality reducing the loss(and many false negatives) keeping the same precision of lower threshold but incrementing the recall significantly. Comparing the other values we can conclude that this was the better cost-quality performance. 

 - For threshold .8 we can see that more votes have been collected(**738** more than base) but not as many as in the previous, this can be expected due to the bigger difference between the initial item probability of being classified and the threshold(%66,6->%80). The longer this difference is, the more the number of votes will be needed to reach it, and higher is the chance of error. 

 - For threshold .9 no votes were collected and this can be expectable given the big gap between the initial classification probability and threshold(%66,6->%90). We can do further analysis to see this case... **DISCUSS if is needed!!!!**

### Error Rate: penalizing false negatives

We add a loss ratio to penalyze the false negatives which we consiser to be more harmful than the false positives, because are those items which are loss forever. With this table we expect to see the performance of the approaches over different classification thresholds, with a loss ratio = 5, expert cost = 20, over 1000 balanced data, 3 initial votes for MV and SMV.

We set a loss ratio in 5 as we think is a conservative value.

<!---
| Approach      | Threshold | Cost   | Error rate | Error rate lr=10 | Precision | Recall |
|---------------|-----------|--------|------------|------------------|-----------|--------|
| Baseline MV   | .5        | 3000   | 0.156      | 0.874            | 0.845     | 0.842  |
| Brown&Allison | .5        | 2377,44| 0.115      | 0.859            | 0.841     | 0.849  |
| SmartStop MV  | .5        | 3000   | 0.158      | 0.855            | 0.843     | 0.841  |
| SmartStop MV  | .6        | 3000   | 0.157      | 0.852            | 0.843     | 0.843  |
| SmartStop MV  | .7        | 4096,16| 0.115      | 0.353            | 0.842     | 0.947  |
| SmartStop MV  | .8        | 3738   | 0.241      | 0.321            | 0.679     | 0.983  |
| SmartStop MV  | .9        | 3000   | 0.298      | 0.371            | 0.629     | 0.984  |

-->



### Performance comparison with other approaches

In this case we compare the cost and quality results against the other approaches, we can expect to see an improvement in quality at a reasonable cost when the smart stop module predict to collect more votes. Given the conservative classification criteria, which is to collect votes to ***exclude*** items and if there is not enough evidence ***include*** them, affects the **precision** because the increment in **false positives(FP)**, following a binary classification terminology we can describe them as those items which real value is ***0*** and are classified as ***1***.

**Behaviour of the approaches. Simulations for unbalanced data 10% possitives, smart stop evaluated over different classifications threshold = [.5, .6, .7, .8, .9], with 1000 balanced items, expert cost 20, workers acc mean ≈ 76%, loss ratio = 5.**

As expected the quality was higher when using the module predictions, reducing the loss and incrementing the recall, as an effect of the classification criteria the precision is reduced. In the case of threshold .9 the recall is high because it reduces the false negative amount due to it strictness, but it has a big impact in precision becoming this case the worse one in terms of number of false positives.

## 3.3.2) Experiments with unbalanced data

At this point we are interested in analyzing the cost-quality performance of the approaches over unbalanced data.

### Results for unbalanced data 10% positives - 90% negatives

**Behaviour of the approaches. Simulations for unbalanced data 10% possitives, over different classifications threshold = [.5, .6, .7, .8, .9], with 1000 balanced items, expert cost 20, workers acc mean ≈ 76%, loss ratio = 5.**

We can get similar conclusion with unbalanced data, the threshold .7 continue having the better performance in terms of cost-quality. In particular the bigger thresholds(.8 and .9) are affected significantly in this case compared to balanced dataset, again the reason is the impact of false positives that in this case are more likely to appear.

### 3.3.3) Results for unbalanced data 1% positives - 99% negatives

**Behaviour of the approaches. Simulations for unbalanced data 1% possitives, over different classifications threshold = [.5, .6, .7, .8, .9], with 1000 balanced items, expert cost 20, workers acc mean ≈ 76%, loss ratio = 5.**

This case is similar to the previous one, with the difference of having a bigger difference in precision values of thresholds .8 and .9, and as expected a directly related loss increment.

## Real-world datasets

### Single predicate binary real world datasets

- Database: BarzanMozafari - Items: 1000 - Workers: 83 - Workers Accuracy Avg: 68.91
- Database: RTE - Items: 800 - Workers: 164 - Workers Accuracy Avg: 83.7
- Database: SpamCF - Items: 101 - Workers: 150 - Workers Accuracy Avg: 69.07
- Database: TEMP - Items: 462 - Workers: 76 - Workers Accuracy Avg: 84.13
- Database: WVSCM - Items: 160 - Workers: 17 - Workers Accuracy Avg: 65.77

In [9]:
def run_experiment_real_world(generate_votes_fn, decision_function, cf, workers_accuracy):
    
    main_results = []
    
    for ct in cts:
        ct = round(ct, 2) #limit to two decimals
        crowd_cost = []
        ct_loss_mv = []
        ct_recall_mv = []
        ct_precision_mv = []
        
        for _ in range(iterations_per_ct):
            params = {
                'workers_accuracy': workers_accuracy,
                'workers_num': workers_num,
                'items_num': items_num,
                'cost_ratio': cr,
                'votes_per_item': base_votes_per_item,
                'classification_fn': cf
            }
    
            items_classification, total_votes = generate_votes_fn(params, items_num, ct, ground_truth, df)
            
            loss,  recall, precision = alg_utils.Metrics.compute_metrics(items_classification, ground_truth, lr)
            ct_loss_mv.append(loss)
            ct_recall_mv.append(recall)
            ct_precision_mv.append(precision)     
            
            ct_i_crowd_cost = alg_utils.get_total_cost(total_votes, cr)
            crowd_cost.append(ct_i_crowd_cost)
            
            #print(f'gt: {len(ground_truth)} - votes: {ct_i_crowd_cost}')
        #end for iterations

        main_results.append([lr,
                             cr,
                             str(cf.__name__),
                             str(df.__name__),
                             sum(ground_truth)/len(ground_truth),
                             ct, 
                             round(np.mean(crowd_cost), 3), 
                             round(np.std(crowd_cost), 3),
                             round(np.mean(ct_loss_mv), 3),
                             round(np.std(ct_loss_mv), 3),
                             round(np.mean(ct_recall_mv), 3),
                             round(np.std(ct_recall_mv), 3),
                             round(np.mean(ct_precision_mv), 3),
                             round(np.std(ct_precision_mv), 3)])

        
    
    return main_results


In [46]:
base_votes_per_item = 3
lr = 5
#cts = [.7 , .8 , .9, .95]
cts = [.5]
iterations_per_ct = 1
cost_ratios = [1/20]
classification_fns = [majority_voting]
decision_fns = [decision_function_bayes]

vts = [3, 5, 7, 10]

total_results = []

real_world_columns = ['dataset','items', 'votes'] + pdColumns

datasets = ['BarzanMozafari', 'RTE', 'SpamCF', 'TEMP', 'WVSCM']

c = 2
e = 0.2

for dataset in datasets:
    ground_truth, workers_accuracy = alg_utils.get_real_dataset_data(dataset)
    '''
    ws_accs_aux = {}
    for w_i, w_a in workers_accuracy.items():
        ws_accs_aux[17+w_i] = w_a
        ws_accs_aux[34+w_i] = w_a
        ws_accs_aux[68+w_i] = w_a
        ws_accs_aux[85+w_i] = w_a
    workers_accuracy = {**workers_accuracy, **ws_accs_aux}
    '''
    items_num = len(ground_truth)
    workers_num = len(workers_accuracy)
    '''
    for df in decision_fns:
        decision_function = df
        for cf in classification_fns:
            for cr in cost_ratios:
    '''
    print(f'dataset: {dataset}, decision fn: {decision_function.__name__}, classification fn: {cf.__name__}, cost ratio: {cr}')

    combination_results = run_experiment_real_world(generate_votes_break_tie, decision_function, cf, workers_accuracy)
    for res_a in combination_results:
        total_results.append([dataset, len(ground_truth), base_votes_per_item] + res_a)

    pd.DataFrame(total_results, columns=real_world_columns).to_csv(f'results/real_world/all_break_tie.csv', index=False)


dataset: BarzanMozafari, decision fn: decision_function_bayes, classification fn: majority_voting, cost ratio: 0.05
dataset: RTE, decision fn: decision_function_bayes, classification fn: majority_voting, cost ratio: 0.05
dataset: SpamCF, decision fn: decision_function_bayes, classification fn: majority_voting, cost ratio: 0.05
dataset: TEMP, decision fn: decision_function_bayes, classification fn: majority_voting, cost ratio: 0.05
dataset: WVSCM, decision fn: decision_function_bayes, classification fn: majority_voting, cost ratio: 0.05


In [13]:
base_votes_per_item = 3
lr = 5
cts = [.95]#[.7 , .8 , .9, .95]
iterations_per_ct = 1
cost_ratios = [1/20]
classification_fns = [majority_voting]
decision_fns = [decision_function_bayes]

total_results = []

datasets = ['WVSCM'] #['BarzanMozafari', 'RTE', 'SpamCF', 'TEMP', 'WVSCM']

for dataset in datasets:
    ground_truth, workers_accuracy = alg_utils.get_real_dataset_data(dataset)
    
    ws_accs_aux = {}
    for w_i, w_a in workers_accuracy.items():
        ws_accs_aux[17+w_i] = w_a
        ws_accs_aux[34+w_i] = w_a
        ws_accs_aux[68+w_i] = w_a
        ws_accs_aux[85+w_i] = w_a
    workers_accuracy = {**workers_accuracy, **ws_accs_aux}
    
    items_num = len(ground_truth)
    workers_num = len(workers_accuracy)
    for df in decision_fns:
        decision_function = df
        for cf in classification_fns:
            for cr in cost_ratios:
                print(f'dataset: {dataset}, decision fn: {decision_function.__name__}, classification fn: {cf.__name__}, cost ratio: {cr}')

                combination_results = run_experiment_real_world(generate_votes_smart_stop, decision_function, cf, workers_accuracy)
                for res_a in combination_results:
                    total_results.append([dataset] + res_a)

                #pd.DataFrame(total_results, columns=pdColumns).to_csv(f'results/real_world/all_results.csv', index=False)


dataset: WVSCM, decision fn: decision_function_bayes, classification fn: majority_voting, cost ratio: 0.05
Items3: 56 - Items40: 0 - ItemsRest: 104


In [11]:
total_results

[['BarzanMozafari',
  5,
  0.05,
  'majority_voting',
  'decision_function_bayes',
  0.491,
  0.95,
  649.55,
  0.0,
  0.14,
  0.0,
  0.955,
  0.0,
  0.94,
  0.0]]

## 4) Next step: Experiments estimating workers accuracy, using Expectation Maximization(EM)

In [40]:
#ground truth data
items_num = 1000
base_votes_per_item = 3

#metrics
lr = 5

#workers
workers_num = 2000 #fixed number
z = 0 #% cheaters
base_workers_acc = .5
fixed_acc = False
fixed_workers_acc = .9

#iterables
cts = [.7 , .8 , .9, .95]
iterations_per_ct = 1

cost_ratios = [1/20]
data_true_balances = [.3, .1]
classification_fns = [majority_voting]#[majority_voting, truth_finder_single]
decision_fns = [decision_function_bayes]#, decision_function_mv]#decision_function_em

total_results = []

print(f'workers: {workers_num}, \n loss ratio: {lr}, \n thresholds: {str(cts)}, \n accuracy: [{str(base_workers_acc)},1], \n iterations per ct: {iterations_per_ct}')

pool = multiprocessing.Pool(processes=5)

for df in decision_fns:
    decision_function = df
    for cf in classification_fns:
        for data_bal in data_true_balances:
            data_true_percentage = data_bal
            ground_truth = alg_utils.generate_gold_data(items_num, data_true_percentage)
            for cr in cost_ratios:
                print(f'decision fn: {decision_function.__name__}, classification fn: {cf.__name__}, data balance: {data_bal}, cost ratio: {cr}')
                
                combination_results = run_experiment_no_multiprocessing(generate_votes_smart_stop, pool, decision_function, cf, data_true_percentage)
                for res_a in combination_results:
                    total_results.append(res_a)
                
                #pd.DataFrame(total_results, columns=pdColumns).to_csv(f'results/simulated_dataset/all_combinations_results_10_20_unbalanced_bayes.csv', index=False)

pool.close()

workers: 2000, 
 loss ratio: 5, 
 thresholds: [0.7, 0.8, 0.9, 0.95], 
 accuracy: [0.5,1], 
 iterations per ct: 1
decision fn: decision_function_bayes, classification fn: majority_voting, data balance: 0.3, cost ratio: 0.05
decision fn: decision_function_bayes, classification fn: majority_voting, data balance: 0.1, cost ratio: 0.05


## Abraham Stopping Algorithm

In this case we implement Abraham algorithm for stopping given two parameters $C$ and $e$ which regulates the stopping criteria
 - Abraham et al., “How Many Workers to Ask?”
 
Logic:
Stop of |Va - Vb| >= C*√t - E*t

Where t = Va + Vb

The paper dont report the best combination of C and E parameters, but shows best performance in $e$= .2 and $e$ = .3

### Experiment over a balanced dataset

In [23]:
def abraham_decision(votes, c = 1, e = 1):
    return {i:abraham_stop_binary(len([x[0] for k,x in i_votes.items() if x[0] == 1]), len([x[0] for k,x in i_votes.items() if x[0] == 0]), c, e) for i, i_votes in votes.items()}


def generate_votes_abraham(params, items_num, ct, gt, df=None):
    total_votes = {}
    
    #base votes
    for i in range(items_num):
        total_votes[i] = {}
        for k in range(params['votes_per_item']):
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)

            total_votes[i][worker_id] = [vote]
            
    #evaluate votes
    results = abraham_decision(total_votes, c, e)
    
    #Check if must continue collecting votes
    items_predicted_classified = alg_utils.get_items_predicted_classified(results)
    must_get_more_votes = len(items_predicted_classified) > 0
     
    while(must_get_more_votes):
        total_votes_aux = {}
        for i, v in items_predicted_classified.items():           
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)
            
            total_votes[i][worker_id] = [vote]
            
            total_votes_aux[i] = total_votes[i]
        #end for
        
        results = abraham_decision(total_votes, c, e)

        #Stop when there are no more items that can be classified
        items_predicted_classified = alg_utils.get_items_predicted_classified(results)
        must_get_more_votes = len(items_predicted_classified) > 0
    #end while
    
    items_classification = alg_utils.classify_items(total_votes, gt, params['classification_fn'], ct)

    return items_classification, total_votes

In [22]:
lr = 5
cr = 1/20
ground_truth = alg_utils.generate_gold_data(items_num, data_true_percentage)
workers_num = 2000
items_num = 1000
cts = [.5]
cs = [2]
es = [.2]
iterations_per_ct = 50

abraham_columns = ["c", "e"] + pdColumns
data_balances = [.5,.3,.1]
decision_function = abraham_decision
cf = majority_voting

total_results_abraham = []
#Abraham
for c in cs:
    for e in es:
        for data_bal in tqdm(data_balances):
            data_true_percentage = data_bal

            results_abraham = run_experiment_no_multiprocessing(generate_votes_abraham, pool, decision_function, cf, data_true_percentage)
            for res_a in results_abraham:
                total_results_abraham.append([c, e] + res_a)

            pd.DataFrame(total_results_abraham, columns=abraham_columns).to_csv(f'results/abraham/results_c2_e02.csv', index=False)



NameError: name 'data_true_percentage' is not defined

We can a observe that a higher $C$ increments quality and cost. For $e$ = .2 we can observe the best performance when $c$ = 1, and when $e$ = .3 the best $C$ is 2.

## Shortest Multi-Run

In [ ]:
from algorithms.sm_run.ShortestMultiRun.helpers.utils import Generator, Workers
from algorithms.sm_run.ShortestMultiRun.ShortestMultiRun import ShortestMultiRun

'''
z - proportion of cheaters
lr - loss ration, i.e., how much a False Negative is more harmful than a False Positive
votes_per_item - crowd votes per item for base round
worker_tests - number of test questions per worker
theta - overall proportion of positive items
filters_num - number of filters
filters_select - selectivity of filters (probability of applying a filter)
filters_dif - difficulty of filters
iter_num - number of iterations for averaging results
---------------------
FP == False Exclusion
FN == False Inclusion
'''



z = 0.3
items_per_worker = 10
baseround_items = 20  # must be a multiple of items_per_worker
if baseround_items % items_per_worker:
    raise ValueError('baseround_items must be a multiple of items_per_worker')
worker_tests = 5
votes_per_item = 3
lr = 5
theta = 0.3
iter_num = 50
data = []

#filters_num = 2
#items_num = 1000
#filters_select = [0.14, 0.14, 0.28, 0.42]
#filters_dif = [1., 1., 1.1, 0.9]

# S-run algorithm
loss_smrun_list = []
cost_smrun_list = []
rec_sm, pre_sm, f_sm, f_sm = [], [], [], []

filters_dif = []

#gt
# 1 = loneliness
for i in range(len(datasets)):
    
    filters_num = len(datasets[i]['predicates'])
    
    
    if (len(datasets[i]['predicates']) == 1):
        predicate_type = 'single'
        ground_truth = alg_utils.load_data(datasets, i, predicate_type)
        gt_p1 = ground_truth[datasets[i]['predicates'][0]]
        filters_select = [
            sum(gt_p1) / len(gt_p1)
        ]
    else:
        predicate_type = 'multi'
        ground_truth = alg_utils.load_data(datasets, i, predicate_type)
        gt_p1 = ground_truth[datasets[i]['predicates'][0]]
        gt_p2 = ground_truth[datasets[i]['predicates'][1]]
        filters_select = [
            sum(gt_p1) / len(gt_p1),
            sum(gt_p2) / len(gt_p2)
        ] 
    
    
    items_num = len(ground_truth[datasets[i]['predicates'][0]])

    params = {
        'filters_num': filters_num,
        'items_num': items_num,
        'baseround_items': baseround_items,
        'items_per_worker': items_per_worker,
        'votes_per_item': votes_per_item,
        'filters_select': filters_select,
        'filters_dif': filters_dif,
        'worker_tests': worker_tests,
        'lr': lr,
        'stop_score': 100
    }

    for _ in range(iter_num):
        # quiz, generation votes
        workers_accuracy = Workers(worker_tests, z).simulate_workers()
        params.update({'workers_accuracy': workers_accuracy,
                       'ground_truth': None
                       })

        _, ground_truth = Generator(params).generate_votes_gt(items_num)
        params.update({'ground_truth': ground_truth})

        # s-run
        loss_smrun, cost_smrun, rec_sm_, pre_sm_, f_beta_sm = ShortestMultiRun(params).run()
        loss_smrun_list.append(loss_smrun)
        cost_smrun_list.append(cost_smrun)
        rec_sm.append(rec_sm_)
        pre_sm.append(pre_sm_)
        f_sm.append(f_beta_sm)

    data.append([worker_tests, worker_tests, lr, np.mean(loss_smrun_list), np.std(loss_smrun_list),
                 np.mean(cost_smrun_list), np.std(cost_smrun_list), 'Crowd-Ensemble', np.mean(rec_sm),
                 np.std(rec_sm), np.mean(pre_sm), np.std(pre_sm), np.mean(f_sm), np.std(f_sm),
                 baseround_items, items_num, theta, filters_num])

    print('SM-RUN    loss: {:1.3f}, loss_std: {:1.3f}, recall: {:1.2f}, rec_std: {:1.3f}, '
          'price: {:1.2f}, price_std: {:1.2f}, precision: {:1.3f}, f_b: {}'
          .format(np.mean(loss_smrun_list), np.std(loss_smrun_list), np.mean(rec_sm),
                  np.std(rec_sm), np.mean(cost_smrun_list), np.std(cost_smrun_list),
                  np.mean(pre_sm), np.mean(f_sm)))

    dataset_name = datasets[i]['folder']
    pd.DataFrame(data,
                 columns=['worker_tests', 'worker_tests', 'lr', 'loss_mean', 'loss_std', 'price_mean', 'price_std',
                          'algorithm', 'recall', 'recall_std', 'precision', 'precision_std',
                          'f_beta', 'f_beta_std', 'baseround_items', 'total_items', 'theta', 'filters_num']
                 ).to_csv(f'results/sm_run/{dataset_name}_base.csv', index=False)

In [1]:
import numpy as np
import pandas as pd

from algorithms.sm_run.ShortestMultiRun.helpers.utils import Generator, Workers
from algorithms.sm_run.ShortestMultiRun.ShortestMultiRun import ShortestMultiRun

'''
z - proportion of cheaters
lr - loss ration, i.e., how much a False Negative is more harmful than a False Positive
votes_per_item - crowd votes per item for base round
worker_tests - number of test questions per worker
theta - overall proportion of positive items
filters_num - number of filters
filters_select - selectivity of filters (probability of applying a filter)
filters_dif - difficulty of filters
iter_num - number of iterations for averaging results
---------------------
FP == False Exclusion
FN == False Inclusion
'''

if __name__ == '__main__':
    z = 0.3
    items_num = 1000
    items_per_worker = 10
    baseround_items = 20  # must be a multiple of items_per_worker
    if baseround_items % items_per_worker:
        raise ValueError('baseround_items must be a multiple of items_per_worker')
    worker_tests = 5
    votes_per_item = 3
    lr = 5
    filters_num = 4
    theta = 0.3
    filters_select = [0.14, 0.14, 0.28, 0.42]
    filters_dif = [1., 1., 1.1, 0.9]
    iter_num = 50
    data = []
    
    params = {
        'filters_num': filters_num,
        'items_num': items_num,
        'baseround_items': baseround_items,
        'items_per_worker': items_per_worker,
        'votes_per_item': votes_per_item,
        'filters_select': filters_select,
        'filters_dif': filters_dif,
        'worker_tests': worker_tests,
        'lr': lr,
        'stop_score': 100
    }

    # S-run algorithm
    loss_smrun_list = []
    cost_smrun_list = []
    rec_sm, pre_sm, f_sm, f_sm = [], [], [], []
    for _ in range(iter_num):
        # quiz, generation votes
        workers_accuracy = Workers(worker_tests, z).simulate_workers()
        params.update({'workers_accuracy': workers_accuracy,
                       'ground_truth': None
                       })

        _, ground_truth = Generator(params).generate_votes_gt(items_num)
        params.update({'ground_truth': ground_truth})

        # s-run
        loss_smrun, cost_smrun, rec_sm_, pre_sm_, f_beta_sm = ShortestMultiRun(params).run()
        loss_smrun_list.append(loss_smrun)
        cost_smrun_list.append(cost_smrun)
        rec_sm.append(rec_sm_)
        pre_sm.append(pre_sm_)
        f_sm.append(f_beta_sm)

    data.append([worker_tests, worker_tests, lr, np.mean(loss_smrun_list), np.std(loss_smrun_list),
                 np.mean(cost_smrun_list), np.std(cost_smrun_list), 'Crowd-Ensemble', np.mean(rec_sm),
                 np.std(rec_sm), np.mean(pre_sm), np.std(pre_sm), np.mean(f_sm), np.std(f_sm),
                 baseround_items, items_num, theta, filters_num])

    print('SM-RUN    loss: {:1.3f}, loss_std: {:1.3f}, recall: {:1.2f}, rec_std: {:1.3f}, '
          'price: {:1.2f}, price_std: {:1.2f}, precision: {:1.3f}, f_b: {}'
          .format(np.mean(loss_smrun_list), np.std(loss_smrun_list), np.mean(rec_sm),
                  np.std(rec_sm), np.mean(cost_smrun_list), np.std(cost_smrun_list),
                  np.mean(pre_sm), np.mean(f_sm)))

    pd.DataFrame(data,
                 columns=['worker_tests', 'worker_tests', 'lr', 'loss_mean', 'loss_std', 'price_mean', 'price_std',
                          'algorithm', 'recall', 'recall_std', 'precision', 'precision_std',
                          'f_beta', 'f_beta_std', 'baseround_items', 'total_items', 'theta', 'filters_num']
                 ).to_csv('output_smv.csv', index=False)

SM-RUN    loss: 0.271, loss_std: 0.067, recall: 0.73, rec_std: 0.146, price: 11.35, price_std: 1.05, precision: 0.971, f_b: 0.7570017095420997


## CrowdExplorer (Kamar et al.)

In [ ]:
import numpy as np
import helpers.algorithms_utils as alg_utils

action_stop = 0
action_continue = 1

'''
    Input: state object
    Output: Label with more votes. In case of tie, returns the one with lower index number.
'''
def majority_voting(st):
    return st.N.index(max(st.N))

consensus_rule_hyperparameter = majority_voting
cost_hyperparameter = 0.1
utility_hyperparameter = 1

#Object State
class TaskState:
    votes: []
    N: []
    N_num: 0
    V: 0
    V_neg: 0
        
#Sample Model
class SampleModel:
    votes: []
    

def calculate_voi():
    pass
    
def calculate_voi(task_state, prm, horizon):
    for _ in range(100): #until timeout?
        models = sample_models(prm)
        task_state = sample_execution_path(task_state, models, horizon)
        
    return task_state.V - task_state.V_neg
        
            
def sample_execution_path(task_state, models, horizon, t):
    label = None
    if t = horizon:
        label = consensus_rule_hyperparameter(task_state.votes) #consensus rule is hyperparameter
    else:
        next_vote = sample_next_vote(task_votes, models)
        task_votes.append(next_vote)
        label = sample_next_vote(task_votes, models)
    #endif
    task_state.N[label] += 1
    task_state.N_num += 1
    task_state.V_neg = ((max(task_state.N) / task_state.N_num) * utility_hyperparameter) - (t - cost_hyperparameter)
    
    if t < horizon:
        st.V = 
    import numpy as np
import 

action_stop = 0
action_continue = 1

'''
    Input: state object
    Output: Label with more votes. In case of tie, returns the one with lower index number.
'''
def majority_voting(st):
    return st.N.index(max(st.N))

consensus_rule_hyperparameter = majority_voting
cost_hyperparameter = 0.1
utility_hyperparameter = 1

class TaskState:
    votes: []
    N: []
    N_num: 0
    V: 0
    V_neg: 0
    task_id: None
        
    def __init__(self, taskId):
        self.task_id = taskId

    
def calculate_voi():
    pass

def crowd_explorer_decision(tasks, horizon, workers_accuracy):
    prm = [] #models
    actions = []
    for task in tasks:
        task_state = TaskState()
        t = 0
        voi = 1 #initialization
        while(voi > 0 or t != horizon):
            voi = calculate_voi(task_state, prm, horizon, t)
            if voi > 0:
                vote = get_next_worker_vote() #"real" vote
                #add_label(prm, vote)
                task_state.votes.append(vote)
                t += 1 #number of real votes
        #end while
        action = 
        actions.append(action)
    
def calculate_voi(task_state, prm, horizon):
    for _ in range(100): #until timeout?
        models = sample_models(prm)
        task_state = sample_execution_path(task_state, models, horizon)
        
    return task_state.V - task_state.V_neg
        
            
def sample_execution_path(task_state, models, horizon, t):
    label = None
    if t = horizon:
        label = consensus_rule_hyperparameter(task_state.votes) #consensus rule is hyperparameter
    else:
        next_vote = sample_next_vote(task_votes, models)
        task_votes.append(next_vote)
        label = sample_next_vote(task_votes, models)
    #endif
    task_state.N[label] += 1
    task_state.N_num += 1
    task_state.V_neg = ((max(task_state.N) / task_state.N_num) * utility_hyperparameter) - (t - cost_hyperparameter)
    
    if t < horizon:
        st.V = 1
#

#to-do likelihood with Expectation Propagation(Tom Minka)
def get_likelihood(st, label):
    return st.N[label] / len(st.N) #mv for now
    
def get_worker_vote(workers_accuracy, gt, i):
    selected_worker_id = np.random.choice(workers_accuracy)
    worker_acc = workers_accuracy[selected_worker_id][0]

    if np.random.binomial(1, worker_acc):
        vote = gt[i]
    else:
        vote = 1 - gt[i]

    return vote
    
#Main fn
def crowd_explorer_decision(tasks, horizon, workers_accuracy, gt):
    prm = [get_likelihood] #models_L
    actions = []
    for task_id in tasks:
        task_state = TaskState(task_id)
        t = 0
        voi = 1 #initialization
        while(voi > 0 or t != horizon):
            voi = calculate_voi(task_state, prm, horizon, t)
            if voi > 0:
                vote = get_worker_vote(workers_accuracy, gt, i)
                #add_label(prm, vote)
                task_state.votes.append(vote)
                t += 1 #number of real votes
        #end while
        action = 
        actions.append(action)